<a href="https://colab.research.google.com/github/wksmirnowa/antidict/blob/master/Jupyter%20Notebooks/Expressive_forms_classification_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 173kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [0]:
import pandas as pd

In [4]:
data = pd.read_csv('expressive_words_train.txt', names=['words'])
data['words'] = data.words.str.lower()
data['values'] = 1
print(data.sample(20))

            words  values
146    старикашка       1
318   пятистенкой       1
221       соколик       1
405      погодкам       1
153        снежок       1
287       никитос       1
40        катюшка       1
6          кисуля       1
220        светик       1
431    задоринкой       1
16     заколдовка       1
412  господинчики       1
26         детина       1
242      бабулька       1
116        манюся       1
315    крупнячком       1
397    собачундру       1
376    глистовуха       1
207     артемидку       1
293        юрасик       1


In [5]:
data1 = pd.read_csv('zaliznyak.txt', names=['words'])
data1['values'] = 0
print(data1.sample(20))

                  words  values
723             почтена       0
107           комлистой       0
1999         фильтровая       0
2555       облепляющими       0
340          развратней       0
581            рычажкам       0
2182         серебряной       0
714   простругивающееся       0
1837           амиловая       0
681          отопритесь       0
2382        разделяться       0
219    попреисполненнее       0
502      перекатываемою       0
2323     расславившееся       0
954         испрыскивал       0
1392        загружавшем       0
519             погребе       0
2198             башами       0
1617        приращённою       0
1378         лесовозном       0


In [0]:
import gensim
import numpy as np

from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [7]:
!wget https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz

--2020-05-17 09:31:10--  https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz’

araneum_none_fastte 100%[===================>]   2.51G  28.1MB/s    in 92s     

2020-05-17 09:32:42 (27.9 MB/s) - ‘araneum_none_fasttextcbow_300_5_2018.tgz’ saved [2691248108/2691248108]



In [0]:
import tarfile
tar = tarfile.open("araneum_none_fasttextcbow_300_5_2018.tgz", "r")
tar.extractall()

In [0]:
model = gensim.models.KeyedVectors.load("araneum_none_fasttextcbow_300_5_2018.model")

In [10]:
df = pd.concat([data1.sample(600), data], axis=0).sample(frac=1)
print(df.sample(20))

                 words  values
2420        вытираемей       0
817            курочна       0
41            катенька       1
1917      взъедавшимся       0
190            кубковы       0
2094       хромируемой       0
200          архичасто       1
1556  переволочившемся       0
129            наташка       1
360          росиянцем       1
2159         подпорота       0
419          шаромыжки       1
14         замерзайкой       1
187          мальчичек       1
1863        труднейших       0
22            солнышко       1
2678         сродником       0
452          нахвастал       0
379          эпиграмма       0
2054       зазвонившем       0


In [0]:
df['model'] = df.words.apply(lambda x: model[x])

In [0]:
X = np.vstack(df.model.values)
y = df['values'].values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
clf = LogisticRegression().fit(X_train, y_train)

In [15]:
y_pred = clf.predict(X_test)
f1_score(y_test, y_pred)

0.7111111111111111

In [16]:
from sklearn import linear_model
clf1 = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf1.fit(X, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [17]:
y_pred = clf1.predict(X_test)
f1_score(y_test, y_pred)

0.909952606635071

In [0]:
dataset = pd.read_csv('words_only.csv', sep='\t', names=['word'])

In [53]:
%%time
ff = open('expressive.csv', 'w+')

for word in dataset.word:
    if len(word) < 4 or len(word) > 18:
      continue
    res = clf1.predict(model[word].reshape(1,-1))
    if res[0] == 1:
        example = f"{word}\t{res}" + "\n"
        ff.write(example)

CPU times: user 15min 14s, sys: 514 ms, total: 15min 15s
Wall time: 15min 15s


In [55]:
examples = pd.read_csv('expressive.csv', sep='\t', names=['word', 'class'])
examples.sample(15)

,word,class
380827,сипатишное,[1]
32575,онкио,[1]
182547,шампаню,[1]
108669,Дишечка,[1]
452871,страшнеееееее,[1]
151246,легкотя,[1]
72083,жыдкой,[1]
272140,Евродеток,[1]
101342,шлачок,[1]
258240,тряснула,[1]
